In [1]:
import pandas as pd

df_merton = pd.read_csv("merton_calib_panel.csv")
df_merton.head()

FileNotFoundError: [Errno 2] No such file or directory: 'merton_calib_panel.csv'

In [ ]:
gvkey = "100022"   # BMW for example

g = df_merton[df_merton["gvkey"].astype(str) == gvkey].copy()
g = g.sort_values("date")

print(len(g))
g[["date","E","B_drop","r","V","sigma_V"]].head()


3519


3519

In [ ]:
import numpy as np

x = np.log(g["E"].values)        # observed log equity
r = g["r"].values                # risk-free
F = g["B_drop"].values           # debt (strike)
tau = np.ones_like(x) * 1.0      # Huang & Yu fix maturity to 1 year
dt = 1/252 

print(x.shape, r.shape, F.shape, tau.shape)


(3519,) (3519,) (3519,) (3519,)


In [ ]:
hhat = np.log(g["V"].values)              # latent log-asset from classical Merton
sigma0 = float(np.median(g["sigma_V"]))   # initial sigma (paper lets it be random, we start near Merton)
mu0 = 0.3                                 # same prior mean as Huang & Yu
delta0 = 0.01                             # small microstructure noise start

dt = 1/252

drift0 = (mu0 - 0.5 * sigma0**2) * dt
dh = np.diff(hhat)

eps0 = (dh - drift0) / (sigma0 * np.sqrt(dt))
eps0 = np.clip(eps0, -5, 5)               # stabilise sampler

print(sigma0, eps0.shape)

0.0696655251192278 (3518,)


In [ ]:
import pymc as pm
import pytensor.tensor as pt
from pytensor.scan import scan

def norm_cdf(z):
    return 0.5 * (1.0 + pt.erf(z / pt.sqrt(2.0)))

def merton_equity_value(V, F, r, tau, sigma):
    eps = 1e-12
    V = pt.maximum(V, eps)
    tau = pt.maximum(tau, eps)
    sigma = pt.maximum(sigma, eps)

    sqrt_tau = pt.sqrt(tau)
    d1 = (pt.log(V / F) + (r + 0.5 * sigma**2) * tau) / (sigma * sqrt_tau)
    d2 = d1 - sigma * sqrt_tau

    return V * norm_cdf(d1) - F * pt.exp(-r * tau) * norm_cdf(d2)


with pm.Model() as mod1:

    # ---- PRIORS (exact Huang & Yu MOD1)
    mu = pm.Normal("mu", mu=0.3, sigma=2.0)
    sigma = pm.InverseGamma("sigma", alpha=3.0, beta=1e-4)
    delta = pm.InverseGamma("delta", alpha=2.5, beta=0.025)

    # ---- STATE EQUATION (latent log-assets)
    h0 = pm.Normal("h0", mu=hhat[0], sigma=1.0)

    eps = pm.Normal("eps", 0.0, 1.0, shape=len(x)-1)
    
    def step(eps_t, h_prev, mu, sigma):
        drift = (mu - 0.5 * sigma**2) * dt
        return h_prev + drift + sigma * pt.sqrt(dt) * eps_t

    h_path, _ = scan(
    fn=step,
    sequences=[eps],
    outputs_info=[h0],
    non_sequences=[mu, sigma],
)
    h = pt.concatenate([[h0], h_path])

    V_latent = pt.exp(h)

    # ---- OBSERVATION EQUATION
    S_model = merton_equity_value(V_latent, F, r, tau, sigma)
    logS = pt.log(pt.maximum(S_model, 1e-12))

    pm.Normal("x_obs", mu=logS, sigma=delta, observed=x)

/var/folders/78/vd95h6m16cg5q2lv04ynfxc80000gn/T/ipykernel_29447/3526156579.py:37: DeprecationWarning: Scan return signature will change. Updates dict will not be returned, only the first argument. Pass `return_updates=False` to conform to the new API and avoid this warning
  h_path, _ = scan(


In [ ]:
with mod1:
    trace = pm.sample(
        draws=200,        # small test
        tune=100,
        chains=1,
        target_accept=0.95,
        initvals={
            "mu": mu0,
            "sigma": sigma0,
            "delta": delta0,
            "h0": hhat[0],
            "eps": eps0,
        },
        progressbar=True
    )

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [mu, sigma, delta, h0, eps]


/Users/telmosantos/Desktop/Merton_NIGbayesian/.venv/lib/python3.13/site-packages/rich/live.py:260: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 1 chain for 100 tune and 157 draw iterations (100 + 157 draws total) took 1637 seconds.
There were 125 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks
